In [1]:
from pathlib import Path
try:
    from generate.utils.review_dialogues import load_jsonl, review
except ModuleNotFoundError:
    from utils.review_dialogues import load_jsonl, review

base_dir = Path('generate') if Path('generate').exists() else Path('.')
dataset_path = base_dir / 'jsons' / 'dataset_openai_10.jsonl'
records = load_jsonl(dataset_path)
review(records)


Record 1/10
dialogue_id: dlg_000000
scenario: technical_issue
sub_scenario: app does not open
complexity: medium
tags: ['hidden_dissatisfaction']
message_count: 7
--------------------------------------------------------------------------------
outcome: resolved
conflict_level: low
mistakes_present: False
num_mistakes: 0
agent_mistakes_main: []
agent_mistakes_sub: []
hidden_dissatisfaction: True
agent_tone: polite
--------------------------------------------------------------------------------
ground_truth.intent: technical_issue
ground_truth.satisfaction: neutral
ground_truth.quality_score: 4
ground_truth.hidden_dissatisfaction: True
ground_truth.agent_mistakes_main: []
DIALOGUE
--------------------------------------------------------------------------------
01. [client] Hey, I can't get the app to open. It just crashes every time I try.
02. [agent] I'm sorry to hear that you're having trouble with the app. Can you tell me what device you're using?
03. [client] I'm on an iPhone 12, and

In [1]:
import json
from pathlib import Path
try:
    from generate.utils.check_distributions import (
        build_targets_manifest,
        load_jsonl,
        compute_achieved,
        build_comparison,
    )
except ModuleNotFoundError:
    from utils.check_distributions import (
        build_targets_manifest,
        load_jsonl,
        compute_achieved,
        build_comparison,
    )

base_dir = Path('generate') if Path('generate').exists() else Path('.')
json_dir = base_dir / 'jsons'
dataset_path = json_dir / 'dataset_openai_500.jsonl'

targets = build_targets_manifest()
rows = load_jsonl(dataset_path)
achieved = compute_achieved(rows)
comparison = build_comparison(targets, achieved)

(json_dir / 'distribution_targets_manifest_500.json').write_text(json.dumps(targets, ensure_ascii=False, indent=2), encoding='utf-8')
(json_dir / 'dist_check_achieved_500.json').write_text(json.dumps(achieved, ensure_ascii=False, indent=2), encoding='utf-8')
(json_dir / 'dist_check_comparison_500.json').write_text(json.dumps(comparison, ensure_ascii=False, indent=2), encoding='utf-8')

def print_table(title, rows_dict):
    print(f'\n## {title}')
    item_w = max(24, min(60, max(len(str(k)) for k in rows_dict))) if rows_dict else 24
    print(f"{'item':<{item_w}} | {'desired':>10} | {'achieved':>10} | {'status':^6}")
    print(f"{'-' * item_w}-+-{'-' * 10}-+-{'-' * 10}-+-{'-' * 6}")
    for item, row in rows_dict.items():
        label = str(item)
        if len(label) > item_w:
            label = label[: item_w - 3] + '...'
        print(f"{label:<{item_w}} | {row['desired']:>10.4f} | {row['achieved']:>10.4f} | {row['status']:^6}")

print('Dataset:', dataset_path)
print('Records:', achieved['n_records'])
print('Threshold:', comparison['threshold'])

flat_sections = [
    'scenario',
    'sub_scenario_global',
    'complexity',
    'outcome',
    'conflict_level',
    'mistakes_present',
    'num_mistakes_if_present',
    'hidden_dissatisfaction_given_resolved',
]

for section in flat_sections:
    print_table(section, comparison['comparison'][section])

print('\n## sub_scenario_within_scenario')
for scenario, rows_ in comparison['comparison']['sub_scenario_within_scenario'].items():
    print_table(f'sub_scenario_within_scenario::{scenario}', rows_)


Dataset: jsons\dataset_openai_500.jsonl
Records: 495
Threshold: 0.05

## scenario
item                     |    desired |   achieved | status
-------------------------+------------+------------+-------
account_access           |     0.1500 |     0.1414 |   🟩   
payment_issue            |     0.2500 |     0.2384 |   🟩   
refund_request           |     0.1000 |     0.1111 |   🟩   
tariff_question          |     0.3000 |     0.3030 |   🟩   
technical_issue          |     0.2000 |     0.2061 |   🟩   

## sub_scenario_global
item                                   |    desired |   achieved | status
---------------------------------------+------------+------------+-------
2FA code not received                  |     0.0214 |     0.0202 |   🟩   
account locked                         |     0.0214 |     0.0222 |   🟩   
app does not open                      |     0.0286 |     0.0343 |   🟩   
auto-renewal                           |     0.0429 |     0.0424 |   🟩   
cannot change email           